In [6]:
from src.mod_transformer import Router
import torch

In [14]:
batch_size = 1
seq_len = 16
num_features = 128

input_data = torch.randn(batch_size, seq_len, num_features)

In [71]:
import torch.nn as nn
class Router(nn.Module):
    def __init__(self, top_k, num_hidden):
        super().__init__()
        self.linear = nn.Linear(num_hidden, 1)
        self.top_k = top_k

    def forward(self, x):
        #get scores for each token
        scores = self.linear(x)
        #get top k tokens 
        top_k_scores, top_k_indices = torch.topk(scores, self.top_k, dim=1)
        #get topk from x
        x_top_k = x.gather(1, top_k_indices.expand(-1, -1, x.size(-1)))

        return x_top_k, top_k_indices

In [72]:
router = Router(top_k=8, num_hidden=128)
router(input_data).shape

torch.Size([1, 8, 128])